# <center> <h1>Caracterizar clientes Grupo empresarial EFPE </h1> </center>


<div style="text-align: center;">
    <img src="../imagenes/Imagen.jpeg" style="width: 30%;">
</div>


<div style="text-align: justify;">
    <p> Con el propósito de optimizar la estrategia comercial, y obtener resultados robustos, se lleva a cabo análisis de segmentación y caracterización de la encuesta recolectada en el presente estudio. El enfoque de esta sección está enfocado a obtener insights mediante modelos matemáticos identificación de patrones y comportamientos únicos presentes en la audiencia. Este proceso se traducirá en la obtención de conocimientos profundos respaldados por datos cuantitativos, lo que a su vez nos capacitará para una comprensión más sólida de las preferencias, necesidades y perfiles demográficos de los clientes.

# espacio para dettalar o incertar cuadro de variables relevantess

In [27]:
# importar librerías necesarias para el análisis econométrico
import pandas as pd
from matplotlib import pyplot as ptl
from copy import deepcopy
import numpy as np
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report

In [28]:
# importar data
data = pd.read_csv('../data/data.csv', sep = ';', encoding='latin1')

## Data transformation

In [29]:
data_copy = deepcopy(data)

In [30]:
# Eliminar variables innecesarias para el análisis
data.drop([
    'Marca temporal', '3. Si seleccionó otra, especifique cúal',
    '9.  ¿De tener experiencia tomando alguno de nuestros cursos qué le cambiaría, a la plataforma o a las clases?',
    '11. Nombre', '12. Celular o correo electrónico. ', '14. Ciudad', 'Dirección de correo electrónico'
], axis = 1, inplace = True);

In [31]:
# Renombrar las variables a fin de facilitar su manejo
data = data.rename(columns= {
    '1. ¿Está dispuesto a tomar cursos de educación en linea?, (1 es poco probable y 5 es muy probable)':'dispu_cursos',
    '2.  Seleccione una o más areas del conocimiento que está interesado':'area_conoci',
    '4. ¿En qué red social confía más, a la hora de recibir publicidad y promociones de productos educativos?  Seleccione varias si lo considera.':'r_social',
    '5. ¿Cuánto sería el valor máximo que está dispuesto a pagar por una licencia mensual? (Acceso ilimitado a todos los cursos)':'valor_mes',
    '6. ¿Cuánto estaría dispuesto a pagar por una licencia anual? (Misma condición anterior)': 'valor_año',
    '7. ¿De haber tomado alguno de nuestros cursos, que tan interesad@ está en seguir capacitándose con nosotros? (Donde 1 es poco y 5 es muy interesado)': 'seguir_efpe',
    '8.  ¿Cuál es la probabilidad de que recomiende nuestra empresa a amigos o familiares?': 'y',
    '10. ¿En qué nivel educativo se encuentra?': 'educacion',
    '13. Edad': 'edad',
    '15. Género': 'genero'  
})

## Encoding features

In [32]:
# Mostrar los primeros 5 rows
data.head(3)

,dispu_cursos,area_conoci,r_social,valor_mes,valor_año,seguir_efpe,y,educacion,edad,genero
0,5,Ciencias políticas,Facebook,mes 100 a 200,año 500 Y 600,4.0,Alta,Profesional,28,Masculino
1,5,Administración pública,Facebook,mes 100 a 200,año 500 Y 600,NaN,Alta,Profesional,26,Femenino
2,3,Ciencias políticas,WhatsApp,mes 50 y 100,año 500 Y 600,2.0,Media,Bachiller,26,Masculino


In [33]:
# Transfromar variable objetivo

# remplazar nan como 1
data['seguir_efpe'] = data['seguir_efpe'].fillna(1)

# tomar los valores nan de la variable objetivo como Baja
data['y'] = data['y'].fillna('Baja')
# tomar los valores nan de la variable objetivo como Baja
data['y'] = data['y'].fillna('Baja')

# 1 si la probabilidad es alta en adquirir la subscription o si no
data['y'].replace({'Alta':1, 'Media':0, 'Baja':0}, inplace = True) 

# convertir a in variable pago maximo mes
data['valor_mes'].replace({'Entre 100 y 200 mil pesos':1, 'Media':0, 'Baja':0}, inplace = True) 

In [34]:
data.head(2)

,dispu_cursos,area_conoci,r_social,valor_mes,valor_año,seguir_efpe,y,educacion,edad,genero
0,5,Ciencias políticas,Facebook,mes 100 a 200,año 500 Y 600,4.0,1,Profesional,28,Masculino
1,5,Administración pública,Facebook,mes 100 a 200,año 500 Y 600,1.0,1,Profesional,26,Femenino


In [35]:
# Crear variable dummys

area_conoci = pd.get_dummies(data['area_conoci'], drop_first= True)
r_social = pd.get_dummies(data['r_social'], drop_first= True)
valor_mes = pd.get_dummies(data['valor_mes'], drop_first= True)
valor_año = pd.get_dummies(data['valor_año'], drop_first= True)
educacion = pd.get_dummies(data['educacion'], drop_first= True)
genero = pd.get_dummies(data['genero'], drop_first= True)

data.drop( [
    'area_conoci', 'r_social', 'valor_mes', 'valor_año', 'educacion', 'genero'
    ], axis= 1, inplace= True)
    
# mostar data transformada
data =pd.concat([ data, area_conoci, r_social, valor_mes, valor_año, educacion, genero  ], axis= 1)
print(data.shape)
data.head(2)

(101, 21)


,dispu_cursos,seguir_efpe,y,edad,Ciencia de datos,Ciencias políticas,Contratación Estatal,Finanzas,Finca Raíz,Otro,...,Me es indiferente,WhatsApp,YouTube,mes 300 y 400,mes 50 y 100,año 600 y 700,año 700 y 800,Postgrado,Profesional,Masculino
0,5,4.0,1,28,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
1,5,1.0,1,26,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [36]:
# crear train y test
y = data.y
X = data.drop(['y'], axis= 1)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state=99)
modelo = LogisticRegression(class_weight= "balanced", max_iter=1000)
modelo.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [39]:
# si no se define queda de .5
prediccion_modelo = modelo.predict(X_test)
prediccion_modelo

array([0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
      dtype=int64)

In [40]:
# Evaluar modelo
print( classification_report(y_test, prediccion_modelo))

              precision    recall  f1-score   support

           0       0.64      0.78      0.70         9
           1       0.80      0.67      0.73        12

    accuracy                           0.71        21
   macro avg       0.72      0.72      0.71        21
weighted avg       0.73      0.71      0.72        21

